In [63]:
import etl_project_gdp as etl
import pandas as pd

In [64]:
list_of_gdp = etl.main()

[Extract] Found the target table. Parsing data...
[Extract] Extracted 209 rows.
[Extract] Saving data to data/Countries_by_GDP.json...
[Extract] Data successfully saved to data/Countries_by_GDP.json.
[ETL] Process completed successfully.
[Transform] Parsing HTML data...


In [65]:
list_of_gdp[list_of_gdp['gdp'] >= 100]

,country,type,gdp,year
0,United States,IMF,30337.16,2025
1,China,IMF,19534.89,2025
2,Germany,IMF,4921.56,2025
3,Japan,IMF,4389.33,2025
4,India,IMF,4271.92,2025
...,...,...,...,...
67,Uzbekistan,IMF,112.65,2024
68,Guatemala,IMF,112.37,2024
69,Oman,IMF,109.99,2024
70,Bulgaria,IMF,108.42,2024


In [67]:
regions = pd.read_json('data/cultural-geo-mapper.json')
merged = list_of_gdp.merge(regions, on='country', how='left')

def top_n_mean(group, n=5):
    # 그룹 내 GDP 상위 n개 선택
    top_n = group.nlargest(n, 'gdp')
    return top_n['gdp'].mean()

merged.groupby('region').apply(top_n_mean, n=5).sort_values(ascending=False)

/var/folders/81/7j06ccbd657bxg70hxvk8d180000gn/T/ipykernel_53814/3560080936.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged.groupby('region').apply(top_n_mean, n=5).sort_values(ascending=False)


region
North America         8622.340
East Asia             5421.570
Western Europe        3244.486
Eastern Europe        1066.600
South Asia            1043.380
Latin America          791.566
Southeast Asia         711.890
Middle East            598.134
Oceania                436.658
Sub-Saharan Africa     197.530
North Africa           182.072
Central Asia           106.386
dtype: float64

In [ ]:
import sqlite3
import pandas as pd
import etl_project_gdp_with_sql as etl_sql

def read_from_sqlite(db_name="data/Countries_by_GDP.db"):
    # SQLite DB 연결
    conn = sqlite3.connect(db_name)
    
    # SQL 쿼리로 테이블 읽기
    query = "SELECT * FROM Countries_by_GDP"
    list_of_gdp = pd.read_sql_query(query, conn)
    
    # 연결 닫기
    conn.close()
    
    return list_of_gdp

def analyze_gdp(list_of_gdp):
    # GDP_USD_billion이 100 이상인 데이터 필터링
    filtered_data = list_of_gdp[list_of_gdp['GDP_USD_billion'] >= 100]
    return filtered_data

list_of_gdp = read_from_sqlite()


In [60]:
with sqlite3.connect("data/Countries_by_GDP.db") as conn:
    cursor = conn.cursor()

    query = '''
        SELECT *
        FROM Countries_by_GDP
        WHERE GDP_USD_billion >= 100;
    '''
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    conn.rollback()

    cursor.close()

('United States', 'IMF', 30337.16, '2025', 'North America')
('China', 'IMF', 19534.89, '2025', 'East Asia')
('Germany', 'IMF', 4921.56, '2025', 'Western Europe')
('Japan', 'IMF', 4389.33, '2025', 'East Asia')
('India', 'IMF', 4271.92, '2025', 'South Asia')
('United Kingdom', 'IMF', 3730.26, '2025', 'Western Europe')
('France', 'IMF', 3283.43, '2025', 'Western Europe')
('Italy', 'IMF', 2459.6, '2025', 'Western Europe')
('Canada', 'IMF', 2330.31, '2025', 'North America')
('Brazil', 'IMF', 2307.16, '2025', 'Latin America')
('Russia', 'IMF', 2195.71, '2025', 'Eastern Europe')
('South Korea', 'IMF', 1947.13, '2025', 'East Asia')
('Australia', 'IMF', 1881.14, '2025', 'Oceania')
('Spain', 'IMF', 1827.58, '2025', 'Western Europe')
('Mexico', 'IMF', 1817.82, '2025', 'North America')
('Indonesia', 'IMF', 1492.62, '2025', 'Southeast Asia')
('Turkey', 'IMF', 1455.41, '2025', 'Eastern Europe')
('Netherlands', 'IMF', 1272.96, '2025', 'Western Europe')
('Saudi Arabia', 'IMF', 1136.58, '2025', 'Middle

In [61]:
with sqlite3.connect("data/Countries_by_GDP.db") as conn:
    cursor = conn.cursor()

    query = '''
        SELECT
        Region,
        ROUND(AVG(GDP_USD_billion), 2) AS Average_GDP_USD_billion
    FROM (
        SELECT
            Region,
            GDP_USD_billion,
            ROW_NUMBER() OVER (PARTITION BY Region ORDER BY GDP_USD_billion DESC) AS Rank
        FROM
            Countries_by_GDP
    )
    WHERE
        Rank <= 5
    GROUP BY
        Region
    ORDER BY
        Average_GDP_USD_billion DESC;
    '''
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    conn.rollback()

    cursor.close()

('North America', 8622.34)
('East Asia', 5421.57)
('Western Europe', 3244.49)
('Eastern Europe', 1066.6)
('South Asia', 1043.38)
('Latin America', 791.57)
('Southeast Asia', 711.89)
('Middle East', 598.13)
('Oceania', 436.66)
('Sub-Saharan Africa', 197.53)
('North Africa', 182.07)
('Central Asia', 106.39)
